In [ ]:
import pandas as pd

# Define file paths
test_file_path = '~/mol/data/handwriting/test.csv'
train_file_path = '~/mol/data/handwriting/train.csv'

# Load the CSV files into pandas DataFrames
test_df = pd.read_csv(test_file_path)
train_df = pd.read_csv(train_file_path)

# Display the first few rows of each DataFrame

print("\nTrain DataFrame:")
train_df.head()

In [ ]:
train_df.columns

In [ ]:
print("Test DataFrame:")
test_df.head()

# settin up spark and hardoop


In [ ]:
pip install pyspark

# importing data iinto sark 

In [ ]:
from pyspark.sql import SparkSession
import os

# Initialize Spark session
spark = SparkSession.builder \
        .appName("Data Loading and Exploration") \
                .config("spark.executor.memory", "4g") \
                                    .getOrCreate()


# Define file paths
test_file_path = '/home/dragon/mol/data/handwriting/test.csv'
train_file_path = '/home/dragon/mol/data/handwriting/train.csv'

# Load the CSV files into Spark DataFrames
test_df = spark.read.csv(test_file_path, header=True, inferSchema=True)
train_df = spark.read.csv(train_file_path, header=True, inferSchema=True)

# Show the first few rows of each DataFrame
print("\nTest DataFrame:")
test_df.show()

print("\nTrain DataFrame:")
train_df.show()




# Inspect the Dataset:
 Examine the structure of the data files and the schema of
the data.
Summary Statistics: Calculate summary statistics using distributed data
processing operations in Spark or MapReduce.

In [ ]:
from pyspark.sql import SparkSession

train_df = train_df.limit(30)
test_df = train_df.limit(30)
# Show the first few rows and the schema of each DataFrame
print("\nTest DataFrame - Schema:")
test_df.printSchema()


print("\nTrain DataFrame - Schema:")
train_df.printSchema()

# Summary statistics
print("\nSummary Statistics for Test DataFrame:")
#test_df.describe().show()

print("\nSummary Statistics for Train DataFrame:")
#train_df.describe().show()


# missisng values

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import StandardScaler, VectorAssembler

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Data Processing Pipeline") \
    .getOrCreate()

# File paths
train_file_path = '/home/dragon/mol/data/handwriting/train.csv'
output_file_path = '/home/dragon/mol/data/handwriting/processed_train.csv'

# Load the train dataset
train_df = spark.read.csv(train_file_path, header=True, inferSchema=True)

# Handle missing values (drop rows with missing values for simplicity)
train_df = train_df.dropna()

# Feature Engineering: Example of generating a new feature
#train_df = train_df.withColumn("new_feature", col("existing_feature") * 2)

# Normalize numerical columns
numeric_cols = [col_name for col_name, data_type in train_df.dtypes if data_type == 'double']

assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features")
assembled_train_df = assembler.transform(train_df)

scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)
scaler_model = scaler.fit(assembled_train_df)
scaled_train_df = scaler_model.transform(assembled_train_df)

# Save processed data to a new CSV file
#scaled_train_df.select(["scaled_features", "new_feature"] + numeric_cols).write.csv(output_file_path, header=True)

# Show some rows of the processed DataFrame
scaled_train_df.show(5)

# Stop the Spark session
spark.stop()


# model selcion

In [8]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator


# Create Spark session
spark = SparkSession.builder \
    .appName("CSV Import Example") \
    .getOrCreate()

# Define file paths
test_file_path = '/home/dragon/mol/data/handwriting/test.csv'
train_file_path = '/home/dragon/mol/data/handwriting/train.csv'

# Read CSV files into Spark DataFrames
test_df = spark.read.csv(test_file_path, header=True, inferSchema=True)
train_df = spark.read.csv(train_file_path, header=True, inferSchema=True)

train_df = train_df.limit(30)
test_df = train_df.limit(30)
df = train_df


# Select columns for X (features) and Y (target)
features_columns = df.columns.copy()
features_columns.remove('writer')  # Remove the target column 'writer'

# Handling non-numeric columns
categorical_cols = [col for col in features_columns if df.select(col).distinct().count() <= 20]
numeric_cols = list(set(features_columns) - set(categorical_cols))

indexers = [
    StringIndexer(inputCol=col, outputCol=f"{col}_index", handleInvalid="skip")
    for col in categorical_cols
]

encoder = OneHotEncoder(inputCols=[indexer.getOutputCol() for indexer in indexers],
                        outputCols=[f"{col}_encoded" for col in categorical_cols])

assembler = VectorAssembler(inputCols=numeric_cols + encoder.getOutputCols(), outputCol='features')

# Pipeline stages
pipeline_stages = indexers + [encoder, assembler]

# Apply all transformations in pipeline
pipeline_model = Pipeline(stages=pipeline_stages)
transformed_df = pipeline_model.fit(df).transform(df)

# Split data into training and testing sets
(training_data, test_data) = transformed_df.randomSplit([0.7, 0.3], seed=123)

# Define Random Forest Regressor
rf = RandomForestRegressor(featuresCol='features', labelCol='writer')

# Train the model
model = rf.fit(training_data)

# Make predictions
predictions = model.transform(test_data)

# Evaluate the model
evaluator = RegressionEvaluator(labelCol='writer', predictionCol='prediction', metricName='rmse')
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE) on test data: {rmse}")



# Stop the Spark session
spark.stop()
